In [1]:
import pandas as pd

In [2]:
def ladder_complementation(ladders):
    """Implementation of the Ladder Complementation algorithm.
    
    :param ladders: a list of tupple, [(df, fullmass, ladder_orientation)]
    :return: a pandas DataFrame contains ladder complementation results.
    
    Please note that the result may requires further manual filter.
    """
    df_res = pd.DataFrame()
    for idx, ladder in enumerate(ladders):
        df = ladder[0]
        fullmass = ladder[1]
        print(f'processing fullmass {fullmass}')
        bcr = base_calling_random(df)
        df_res = process_single_ladder(*bcr, fullmass, df_res=df_res, idx=idx)

    return df_res

In [3]:
def process_single_ladder(df, mass_pairs, full_mass, 
                          orientation=5, df_res=pd.DataFrame(), idx=-1):
    seats = int(full_mass // 320)
    col_base = 'Base' if idx < 0 else f'Base{idx}'
    col_mass = 'Mass' if idx < 0 else f'Mass{idx}'
    if df_res.empty:
        df_res = pd.DataFrame()
        df_res['position'] = range(1, seats+1)
        df_res.set_index('position', inplace=True)
    df_res[col_base] = ''
    df_res[col_mass] = ''
    
    df = df.fillna(0)
    for idx, row in df.iterrows():
        if row.Mass <= 0:
            continue
        pos = int(row.Mass // 320)
        if orientation == 3:
            pos = seats + 1 - pos
#             pos -= 1
        mass = df_res.loc[pos, col_mass] 
        if not mass:
            mass = row.Mass
        else:
            mass = '{}, {}'.format(mass, row.Mass)
        df_res.loc[pos, col_mass] = mass
        
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        idx = df_pair['Mass'].idxmax()
        if orientation == 3:
            idx = df_pair['Mass'].idxmin()
        pos = int(df_pair.loc[idx].Mass // 320)
        if orientation == 3:
            pos = seats + 1 - pos
            pos -= 1
        base = df_res.loc[pos, col_base] 
        if not base:
            base = t[2]
        else:
            base = '{}, {}'.format(base, t[2])
        df_res.loc[pos, col_base] = base
    
    return df_res